In [1]:
import numpy as np 
from implementations import *
from helpers import *
from helper_functions import *
from optimization import *

In [2]:
# load the training dataset
print("loading the training dataset")
DATA_TRAIN_PATH = 'train.csv'
y_tr, tx_tr, ids_tr = load_csv_data(DATA_TRAIN_PATH)
print("training dataset loaded")

loading the training dataset
training dataset loaded


In [3]:
# load the test dataset
print("loading the test dataset")
DATA_TEST_PATH = 'test.csv'
y_te, tx_te, ids_te = load_csv_data(DATA_TEST_PATH)
print("test dataset loaded")

loading the test dataset
test dataset loaded


In [4]:
# printing the shapes of the arrays
print("the shape of the training y is ",y_tr.shape," and the training tx is ", tx_tr.shape)
print("the shape of the test y is ",y_te.shape," and the test tx is ",tx_te.shape)

the shape of the training y is  (250000,)  and the training tx is  (250000, 30)
the shape of the test y is  (568238,)  and the test tx is  (568238, 30)


In [5]:
# testing the mean square gd
initial_w = np.zeros(tx_tr.shape[1])
max_iters = 500
gamma = 0.00008

#w, loss = mean_squared_error_gd(y_tr, tx_tr, initial_w, max_iters, gamma)

#print("end of the mean_squared_error_gd with w=",w," and loss=", loss)

In [6]:
# test least square
w, loss = least_squares(y_tr, tx_tr)
print("end of the least_squares with w=",w," and loss=", loss)

end of the least_squares with w= [ 8.03494312e-05 -7.20202273e-03 -6.05417273e-03 -5.47559065e-04
 -1.93874700e-02  4.73451621e-04 -2.60379054e-02  3.25106300e-01
 -3.80780282e-05 -2.72724919e+00 -2.21220140e-01  9.50794091e-02
  6.40351613e-02  2.73550887e+00 -3.31801241e-04 -9.54325120e-04
  2.74026561e+00 -5.34164891e-04  9.73498581e-04  3.69225052e-03
  3.54487449e-04 -5.43344598e-04 -3.30448035e-01 -1.40800498e-03
  8.31432888e-04  1.02117272e-03 -1.68047416e-03 -5.83664818e-03
 -1.11087997e-02  2.72770912e+00]  and loss= 0.3396868094770935


In [7]:
# test mean squared error sgd

initial_w = np.zeros(tx_tr.shape[1])
max_iters = 100
gamma = 0.00001

#w, loss = mean_squared_error_sgd(y_tr, tx_tr, initial_w, max_iters, gamma)

#print("end of the mean_squared_error_sgd with w=",w," and loss=", loss)

In [7]:
# test ridge regression 

#initial_w = np.zeros(tx_tr.shape[1])
#max_iters = 100
#lambda_ = 0.0005

#w, loss = ridge_regression(y_tr, tx_tr, lambda_)
lambda_ = ridge_regression_best_lambda(y_tr, tx_tr)
print("best lambda is ",lambda_)
w, loss = ridge_regression(y_tr, tx_tr, lambda_)

#y_pred = predict(w, tx_te)
#y_pred[y_pred==0] = -1

#OUTPUT_PATH = 'sample-submission'
#create_csv_submission(ids_te, y_pred, OUTPUT_PATH)

print("end of the ridge_regression with w=",w," and loss=", loss)

Current lambda=0.0001, the loss=0.33968798676609097
Current lambda=0.00013738237958832623, the loss=0.33968803954866844
Current lambda=0.00018873918221350977, the loss=0.33968813859041375
Current lambda=0.0002592943797404667, the loss=0.3396883245147402
Current lambda=0.0003562247890262444, the loss=0.3396886731920951
Current lambda=0.0004893900918477494, the loss=0.3396893258059021
Current lambda=0.0006723357536499335, the loss=0.33969054373021085
Current lambda=0.0009236708571873865, the loss=0.33969280743924646
Current lambda=0.0012689610031679222, the loss=0.3396969916896993
Current lambda=0.0017433288221999873, the loss=0.33970466833847507
Current lambda=0.002395026619987486, the loss=0.3397186122395501
Current lambda=0.0032903445623126675, the loss=0.33974360637308687
Current lambda=0.004520353656360241, the loss=0.33978763536530615
Current lambda=0.006210169418915616, the loss=0.3398634706893294
Current lambda=0.008531678524172805, the loss=0.33999040783666146
Current lambda=0.0

In [5]:
# test logistic regression

initial_w = np.zeros(tx_tr.shape[1])
# initial_w = w
max_iters = 200
gamma = 0.000001

w, loss = logistic_regression(y_tr, tx_tr, initial_w, max_iters, gamma)

print("end of the logistic_regression with w=",w," and loss=", loss)

Preiteration, the loss=265618.8699766104, the grad=-170.99750063180275
Current iteration=0, the loss=-131622.40316858073, the grad=1821.1184651771323
Current iteration=1, the loss=-304224.27345602674, the grad=834.2998806025661
Current iteration=2, the loss=-473819.023226236, the grad=822.0329349913256
Current iteration=3, the loss=-642832.2535794436, the grad=819.95442066089
Current iteration=4, the loss=-811580.6865605841, the grad=819.1344796372504
Current iteration=5, the loss=-980135.6385100734, the grad=818.555115367976
Current iteration=6, the loss=-1148531.4801186854, the grad=818.0741678461544
Current iteration=7, the loss=-1316792.7929266673, the grad=817.6605291273098
Current iteration=8, the loss=-1484939.014645783, the grad=817.3002315478669
Current iteration=9, the loss=-1652985.9149293667, the grad=816.9837174833692
Current iteration=10, the loss=-1820946.4446302052, the grad=816.7036473347346
Current iteration=11, the loss=-1988831.3445494724, the grad=816.4542210787805

/home/ml-project-1-ahl/helper_functions.py:61: RuntimeWarning: divide by zero encountered in log
  return -np.sum(y * np.log(sigmoid(v)) + (1 - y) * np.log(1-sigmoid(v)))


Current iteration=255, the loss=-inf, the grad=812.7656040071687
Current iteration=256, the loss=-inf, the grad=812.7649808796759


/home/ml-project-1-ahl/implementations.py:180: RuntimeWarning: invalid value encountered in double_scalars
  if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:


Current iteration=257, the loss=-inf, the grad=812.764364029257
Current iteration=258, the loss=-inf, the grad=812.7637533727523
Current iteration=259, the loss=-inf, the grad=812.763148828363
Current iteration=260, the loss=-inf, the grad=812.7625503156247
Current iteration=261, the loss=-inf, the grad=812.7619577553808
Current iteration=262, the loss=-inf, the grad=812.761371069756
Current iteration=263, the loss=-inf, the grad=812.7607901821353
Current iteration=264, the loss=-inf, the grad=812.7602150171369
Current iteration=265, the loss=-inf, the grad=812.7596455005887
Current iteration=266, the loss=-inf, the grad=812.7590815595062
Current iteration=267, the loss=-inf, the grad=812.7585231220698


/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Current iteration=268, the loss=nan, the grad=812.7579701176023
Current iteration=269, the loss=nan, the grad=812.7574224765474
Current iteration=270, the loss=nan, the grad=812.7568801304486
Current iteration=271, the loss=nan, the grad=812.7563430119293
Current iteration=272, the loss=nan, the grad=812.7558110546707
Current iteration=273, the loss=nan, the grad=812.7552841933941
Current iteration=274, the loss=nan, the grad=812.7547623638405
Current iteration=275, the loss=nan, the grad=812.7542455027514
Current iteration=276, the loss=nan, the grad=812.7537335478511
Current iteration=277, the loss=nan, the grad=812.7532264378285
Current iteration=278, the loss=nan, the grad=812.752724112318
Current iteration=279, the loss=nan, the grad=812.7522265118845
Current iteration=280, the loss=nan, the grad=812.7517335780036
Current iteration=281, the loss=nan, the grad=812.7512452530484
Current iteration=282, the loss=nan, the grad=812.7507614802694
Current iteration=283, the loss=nan, the 